<a href="https://colab.research.google.com/github/CaoCharles/Deep-Learning-with-PyTorch/blob/master/1_ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
# 匯入套件
import pandas as pd
from glob import glob
import os
from shutil import copyfile
from torch.utils.data import Dataset
from PIL import Image
import numpy as np
from numpy.random import permutation
import matplotlib.pyplot as plt
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18,resnet34
from torchvision.models.inception import inception_v3
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import pickle
%matplotlib inline

In [0]:
!pip install Pillow==4.1.1
!pip install PIL
!pip install image
import PIL.Image

  Could not find a version that satisfies the requirement PIL (from versions: )
No matching distribution found for PIL


In [0]:
# 殘差神經網路基本架構

class ResNetBasicBlock(nn.Module):
    
    def __init__(self,in_channels,out_channels,stride):

        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride = stride, padding = 1, bias = False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = stride, padding = 1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.stride = stride
        
    def forward(self,x):
        
        residual = x
        out = self.conv1(x)
        out = F.relu(self.bn1(out), inplace = True)
        out = self.conv2(out)
        out = self.bn2(out)
        out += residual
        return F.relu(out)  

In [0]:
from torchvision.models import resnet18

resnet = resnet18(pretrained = False)

resnet

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [0]:
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# 資料轉換
from torchvision import transforms
from torchvision.datasets import ImageFolder

data_transform = transforms.Compose([
        transforms.Resize((299,299)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
# For Dogs & Cats dataset

train_dset = ImageFolder('/content/drive/My Drive/Colab Notebooks/dogsandcats/train',transform = data_transform)
val_dset = ImageFolder('/content/drive/My Drive/Colab Notebooks/dogsandcats/test',transform = data_transform)
classes = 2

In [0]:
train_dset

Dataset ImageFolder
    Number of datapoints: 22500
    Root Location: /content/drive/My Drive/Colab Notebooks/dogsandcats/train
    Transforms (if any): Compose(
                             Resize(size=(299, 299), interpolation=PIL.Image.BILINEAR)
                             ToTensor()
                             Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                         )
    Target Transforms (if any): None

In [0]:
val_dset

Dataset ImageFolder
    Number of datapoints: 2500
    Root Location: /content/drive/My Drive/Colab Notebooks/dogsandcats/test
    Transforms (if any): Compose(
                             Resize(size=(299, 299), interpolation=PIL.Image.BILINEAR)
                             ToTensor()
                             Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                         )
    Target Transforms (if any): None

In [0]:
# 訓練集與驗證集

from torch.utils.data import DataLoader

train_loader = DataLoader(train_dset,batch_size=32,shuffle=False,num_workers=3)
val_loader = DataLoader(val_dset,batch_size=32,shuffle=False,num_workers=3)


In [0]:
#Create ResNet model

import torch
from torchvision.models import resnet34

my_resnet = resnet34(pretrained=True)


is_cuda = torch.cuda.is_available()

is_cuda

True

In [0]:
list(my_resnet.children())

[Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False),
 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 ReLU(inplace),
 MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False),
 Sequential(
   (0): BasicBlock(
     (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (relu): ReLU(inplace)
     (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   )
   (1): BasicBlock(
     (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (relu): ReLU(inplace)
     (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bi

In [0]:
# 扣除網路架構最後的AvgPool2D
list(my_resnet.children())[:-1]

[Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False),
 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 ReLU(inplace),
 MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False),
 Sequential(
   (0): BasicBlock(
     (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (relu): ReLU(inplace)
     (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   )
   (1): BasicBlock(
     (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (relu): ReLU(inplace)
     (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bi

In [0]:
if is_cuda:
    my_resnet = my_resnet.cuda()

my_resnet = nn.Sequential(*list(my_resnet.children())[:-1])
my_resnet

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, ker

In [0]:
!pip install torchsummary

In [0]:
from torchsummary import summary

summary(my_resnet, (3, 299, 299))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 150, 150]           9,408
       BatchNorm2d-2         [-1, 64, 150, 150]             128
              ReLU-3         [-1, 64, 150, 150]               0
         MaxPool2d-4           [-1, 64, 75, 75]               0
            Conv2d-5           [-1, 64, 75, 75]          36,864
       BatchNorm2d-6           [-1, 64, 75, 75]             128
              ReLU-7           [-1, 64, 75, 75]               0
            Conv2d-8           [-1, 64, 75, 75]          36,864
       BatchNorm2d-9           [-1, 64, 75, 75]             128
             ReLU-10           [-1, 64, 75, 75]               0
       BasicBlock-11           [-1, 64, 75, 75]               0
           Conv2d-12           [-1, 64, 75, 75]          36,864
      BatchNorm2d-13           [-1, 64, 75, 75]             128
             ReLU-14           [-1, 64,

In [0]:
list(my_resnet.parameters())

[Parameter containing:
 tensor([[[[ 5.4109e-03, -6.9092e-03,  7.8839e-03,  ...,  4.9072e-02,
             3.0660e-02,  2.5398e-02],
           [ 4.1081e-02,  3.1296e-02,  3.2265e-02,  ...,  3.3145e-02,
             2.9754e-02,  4.1735e-02],
           [ 4.9519e-03, -3.1705e-02, -6.1310e-02,  ..., -9.7493e-02,
            -1.1601e-01, -1.2191e-01],
           ...,
           [-1.2287e-02, -2.4841e-02, -9.3052e-03,  ...,  1.7113e-02,
             2.4631e-03,  1.6726e-02],
           [ 3.9117e-03,  4.4537e-03,  3.6315e-02,  ...,  1.0371e-01,
             7.3973e-02,  5.9085e-02],
           [ 1.6784e-02,  8.8902e-03,  3.1312e-02,  ...,  9.6964e-02,
             8.3749e-02,  9.6970e-02]],
 
          [[-7.7192e-03, -8.7711e-03,  1.4143e-02,  ...,  3.3901e-02,
             2.5483e-02,  2.4275e-02],
           [ 5.3961e-02,  4.4677e-02,  3.4326e-02,  ...,  1.3392e-02,
             1.9135e-02,  3.7995e-02],
           [ 1.0251e-03, -5.4513e-02, -1.0225e-01,  ..., -1.9231e-01,
            -1.9

In [0]:
for p in my_resnet.parameters():
    p.requires_grad = False

In [0]:
#For training data

# Stores the labels of the train data
trn_labels = []

# Stores the pre convoluted features of the train data
trn_features = []

m = nn.Sequential(*list(my_resnet.children())[:-1])

#Iterate through the train data and store the calculated features and the labels
for d,la in train_loader:
    o = m(Variable(d.cuda()))
    o = o.view(o.size(0),-1)
    trn_labels.extend(la)
    trn_features.extend(o.cpu().data)

#For validation data

#Iterate through the validation data and store the calculated features and the labels
val_labels = []
val_features = []
for d,la in val_loader:
    o = m(Variable(d.cuda()))
    o = o.view(o.size(0),-1)
    val_labels.extend(la)
    val_features.extend(o.cpu().data)

In [0]:
np.shape(trn_features[0])

torch.Size([51200])

In [0]:
list(d.cuda())

[tensor([[[-1.8953, -1.8782, -1.8610,  ..., -1.3815, -1.4329, -1.4329],
          [-1.8953, -1.8782, -1.8610,  ..., -1.5185, -1.5528, -1.5528],
          [-1.8782, -1.8782, -1.8610,  ..., -1.6213, -1.6555, -1.6555],
          ...,
          [-1.1760, -1.1075, -0.8678,  ..., -1.2274, -1.2445, -1.2445],
          [-1.1075, -1.1932, -0.9363,  ..., -1.2274, -1.2445, -1.2445],
          [-1.0048, -1.1589, -0.9705,  ..., -1.2274, -1.2445, -1.2445]],
 
         [[-1.7906, -1.7731, -1.7556,  ..., -1.1604, -1.1429, -1.1429],
          [-1.7906, -1.7731, -1.7556,  ..., -1.3004, -1.2829, -1.2829],
          [-1.7731, -1.7731, -1.7556,  ..., -1.4405, -1.4230, -1.4230],
          ...,
          [-0.9503, -0.9153, -0.6702,  ..., -1.1078, -1.1253, -1.1253],
          [-0.8627, -0.9503, -0.6877,  ..., -1.1078, -1.1253, -1.1253],
          [-0.7052, -0.8803, -0.7052,  ..., -1.1078, -1.1253, -1.1253]],
 
         [[-1.6476, -1.6302, -1.6127,  ..., -0.8981, -0.8807, -0.8807],
          [-1.6476, -1.6302,

In [0]:
np.shape(o.cpu().data)

torch.Size([4, 51200])

In [0]:
np.shape(m(Variable(d.cuda())).view(o.size(0),-1))

torch.Size([4, 51200])

In [0]:
class FeaturesDataset(Dataset):
    def __init__(self,featlst,labellst):
        self.featlst = featlst
        self.labellst = labellst
    def __getitem__(self,index):
        return (self.featlst[index],self.labellst[index])
    def __len__(self):
        return len(self.labellst)

In [0]:
#Creating dataset for train and validation
trn_feat_dset = FeaturesDataset(trn_features,trn_labels)
val_feat_dset = FeaturesDataset(val_features,val_labels)

#Creating data loader for train and validation
trn_feat_loader = DataLoader(trn_feat_dset,batch_size=64,shuffle=True)
val_feat_loader = DataLoader(val_feat_dset,batch_size=64)

In [0]:
len(list(trn_feat_dset))

22500

In [0]:
class FullyConnectedModel(nn.Module):
    def __init__(self,in_size,out_size):
        super().__init__()
        self.fc = nn.Linear(in_size,out_size)

    def forward(self,inp):
        out = self.fc(inp)
        return out
        
fc_in_size = 51200

fc = FullyConnectedModel(fc_in_size,classes)

if is_cuda:
    fc = fc.cuda()
    
optimizer = optim.Adam(fc.parameters(),lr=0.0001)

In [0]:
def fit(epoch,model,data_loader,phase='training',volatile=False):
    if phase == 'training':
        model.train()
    if phase == 'validation':
        model.eval()
        volatile=True
    running_loss = 0.0
    running_correct = 0
    for batch_idx , (data,target) in enumerate(data_loader):
        if is_cuda:
            data,target = data.cuda(),target.cuda()
        data , target = Variable(data,volatile),Variable(target)
        if phase == 'training':
            optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output,target)
        
        running_loss += F.cross_entropy(output,target,size_average=False).data[0]
        preds = output.data.max(dim=1,keepdim=True)[1]
        running_correct += preds.eq(target.data.view_as(preds)).cpu().sum()
        if phase == 'training':
            loss.backward()
            optimizer.step()
    
    loss = float(running_loss)/float(len(data_loader.dataset))
    accuracy = 100. * float(running_correct)/float(len(data_loader.dataset))
    
    print(f'{phase} loss is {loss:{5}.{2}} and {phase} accuracy is {float(running_correct)}/{float(len(data_loader.dataset))}{accuracy:{10}.{4}}')
    return loss,accuracy

In [0]:
train_losses , train_accuracy = [],[]
val_losses , val_accuracy = [],[]

for epoch in range(1,10):
    epoch_loss, epoch_accuracy = fit(epoch, fc, trn_feat_loader, phase = 'training')
    val_epoch_loss , val_epoch_accuracy = fit(epoch, fc, val_feat_loader, phase = 'validation')
    train_losses.append(epoch_loss)
    train_accuracy.append(epoch_accuracy)
    val_losses.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


training loss is  0.12 and training accuracy is 21629.0/22500.0     96.13
validation loss is  0.03 and validation accuracy is 2482.0/2500.0     99.28
training loss is 0.012 and training accuracy is 22462.0/22500.0     99.83
validation loss is 0.022 and validation accuracy is 2487.0/2500.0     99.48
training loss is 0.0046 and training accuracy is 22497.0/22500.0     99.99
validation loss is  0.02 and validation accuracy is 2486.0/2500.0     99.44
training loss is 0.0025 and training accuracy is 22500.0/22500.0     100.0
validation loss is 0.018 and validation accuracy is 2488.0/2500.0     99.52
training loss is 0.0016 and training accuracy is 22500.0/22500.0     100.0
validation loss is 0.018 and validation accuracy is 2489.0/2500.0     99.56
training loss is 0.0011 and training accuracy is 22500.0/22500.0     100.0
validation loss is 0.017 and validation accuracy is 2488.0/2500.0     99.52
training loss is 0.00086 and training accuracy is 22500.0/22500.0     100.0
validation loss is 0

In [0]:
list(trn_feat_loader.dataset)

[(tensor([0.0000, 0.0000, 0.3597,  ..., 0.6652, 1.0308, 0.9482]), tensor(0)),
 (tensor([0.0000, 0.0000, 0.0000,  ..., 1.4420, 1.4039, 1.1563]), tensor(0)),
 (tensor([1.7319, 2.6342, 3.2188,  ..., 0.0309, 0.0000, 0.0000]), tensor(0)),
 (tensor([0., 0., 0.,  ..., 0., 0., 0.]), tensor(0)),
 (tensor([0.0000, 0.0000, 0.0000,  ..., 1.8784, 1.5208, 1.5880]), tensor(0)),
 (tensor([0.0000, 1.2067, 2.1333,  ..., 0.7737, 0.2126, 0.0000]), tensor(0)),
 (tensor([0.4233, 0.9153, 0.0000,  ..., 0.0000, 0.0000, 0.0000]), tensor(0)),
 (tensor([0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.1489, 0.3486]), tensor(0)),
 (tensor([0.0000, 0.0000, 0.0917,  ..., 0.5299, 0.0000, 0.0000]), tensor(0)),
 (tensor([0.9094, 0.6631, 0.0000,  ..., 0.0000, 0.0000, 0.0000]), tensor(0)),
 (tensor([0.0000, 0.0000, 0.0000,  ..., 1.7500, 1.0666, 0.6646]), tensor(0)),
 (tensor([0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0229, 0.0000]), tensor(0)),
 (tensor([0.0000, 0.0000, 0.0000,  ..., 2.0256, 2.5680, 1.9986]), tensor(0)),
 (tensor([